In [1]:
import torch
torch.cuda.is_available()


True

In [2]:
import torch
print("GPU available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


GPU available: True
GPU: Tesla T4


In [3]:
!pip install kaggle


In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shaunthesheep/microsoft-catsvsdogs-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'microsoft-catsvsdogs-dataset' dataset.
Path to dataset files: /kaggle/input/microsoft-catsvsdogs-dataset


In [5]:
import os

base_path = path  # from kagglehub
print(os.listdir(base_path))


['PetImages', 'readme[1].txt', 'MSR-LA - 3467.docx']


In [6]:
pet_path = os.path.join(base_path, "PetImages")
print(os.listdir(pet_path))


['Dog', 'Cat']


In [7]:
import shutil
import os

source_dir = pet_path  # this is PetImages
target_dir = "/content/dataset_raw"

shutil.copytree(source_dir, target_dir)

print("Dataset copied to writable directory.")



Dataset copied to writable directory.


In [8]:
from PIL import Image

def clean_images(folder):
    bad_images = 0
    for cls in ["Cat", "Dog"]:
        class_path = os.path.join(folder, cls)
        for img in os.listdir(class_path):
            img_path = os.path.join(class_path, img)

            # Skip non-image system files
            if not img.lower().endswith((".jpg", ".jpeg", ".png")):
                continue

            try:
                Image.open(img_path).verify()
            except:
                os.remove(img_path)
                bad_images += 1

    print(f"Removed {bad_images} corrupted images")

clean_images("/content/dataset_raw")


/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Removed 2 corrupted images


In [9]:
import torch
import torch.nn as nn

class CustomCNN(nn.Module):
    def __init__(self):
        super().__init__()

        # -------- Convolution Block 1 --------
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25)
        )

        # -------- Convolution Block 2 --------
        self.block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25)
        )

        # -------- Convolution Block 3 --------
        self.block3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25)
        )

        # -------- Classifier --------
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 2)
        )

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.classifier(x)
        return x


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomCNN().to(device)

print(model)


CustomCNN(
  (block1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.25, inplace=False)
  )
  (block2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.25, inplace=False)
  )
  (block3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.25, inplace=Fal

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [12]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [14]:
import os, shutil, random

SOURCE_DIR = "/content/dataset_raw"   # cleaned data
TARGET_DIR = "/content/dataset"       # new structure

# Create directories
for split in ["train", "val"]:
    for cls in ["cat", "dog"]:
        os.makedirs(f"{TARGET_DIR}/{split}/{cls}", exist_ok=True)

print("Folders created.")



Folders created.


In [15]:
def split_and_copy(class_name):
    src = os.path.join(SOURCE_DIR, class_name.capitalize())
    images = os.listdir(src)
    random.shuffle(images)

    split_idx = int(0.8 * len(images))
    train_imgs = images[:split_idx]
    val_imgs = images[split_idx:]

    for img in train_imgs:
        shutil.copy(
            os.path.join(src, img),
            f"{TARGET_DIR}/train/{class_name}/{img}"
        )

    for img in val_imgs:
        shutil.copy(
            os.path.join(src, img),
            f"{TARGET_DIR}/val/{class_name}/{img}"
        )

split_and_copy("cat")
split_and_copy("dog")

print("Train/Validation split completed.")


Train/Validation split completed.


In [16]:
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder("dataset/train", transform=train_transform)
val_data = datasets.ImageFolder("dataset/val", transform=val_transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

print("Classes:", train_data.classes)


Classes: ['cat', 'dog']


In [17]:
model = CustomCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [18]:
def train_one_epoch(model, loader):
    model.train()
    total_loss = 0
    correct = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()

    avg_loss = total_loss / len(loader)
    accuracy = correct / len(loader.dataset)

    return avg_loss, accuracy


In [19]:
def validate(model, loader):
    model.eval()
    correct = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            correct += (outputs.argmax(1) == labels).sum().item()

    accuracy = correct / len(loader.dataset)
    return accuracy


In [20]:
epochs = 15

for epoch in range(epochs):
    loss, train_acc = train_one_epoch(model, train_loader)
    val_acc = validate(model, val_loader)

    print(
        f"Epoch {epoch+1}/{epochs} | "
        f"Loss: {loss:.4f} | "
        f"Train Acc: {train_acc:.4f} | "
        f"Val Acc: {val_acc:.4f}"
    )


/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch 1/15 | Loss: 1.0359 | Train Acc: 0.5626 | Val Acc: 0.6174
Epoch 2/15 | Loss: 0.6542 | Train Acc: 0.5988 | Val Acc: 0.6370
Epoch 3/15 | Loss: 0.6311 | Train Acc: 0.6189 | Val Acc: 0.6554
Epoch 4/15 | Loss: 0.6170 | Train Acc: 0.6440 | Val Acc: 0.6908
Epoch 5/15 | Loss: 0.5991 | Train Acc: 0.6634 | Val Acc: 0.7114
Epoch 6/15 | Loss: 0.5772 | Train Acc: 0.6865 | Val Acc: 0.6496
Epoch 7/15 | Loss: 0.5515 | Train Acc: 0.7091 | Val Acc: 0.7666
Epoch 8/15 | Loss: 0.5234 | Train Acc: 0.7375 | Val Acc: 0.7796
Epoch 9/15 | Loss: 0.4958 | Train Acc: 0.7619 | Val Acc: 0.7868
Epoch 10/15 | Loss: 0.4763 | Train Acc: 0.7759 | Val Acc: 0.8176
Epoch 11/15 | Loss: 0.4590 | Train Acc: 0.7838 | Val Acc: 0.7166
Epoch 12/15 | Loss: 0.4398 | Train Acc: 0.7966 | Val Acc: 0.7968
Epoch 13/15 | Loss: 0.4337 | Train Acc: 0.7998 | Val Acc: 0.8360
Epoch 14/15 | Loss: 0.4157 | Train Acc: 0.8101 | Val Acc: 0.8240
Epoch 15/15 | Loss: 0.4131 | Train Acc: 0.8157 | Val Acc: 0.8340


In [21]:
def final_evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    return accuracy

In [22]:
final_val_acc = final_evaluate(model, val_loader)
print(f"Final Validation Accuracy: {final_val_acc:.4f}")


Final Validation Accuracy: 0.8340


In [23]:
def test_model(model, loader):
    model.eval()  # switch to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # disable gradients
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            preds = outputs.argmax(1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    return accuracy


In [24]:
final_test_accuracy = test_model(model, val_loader)
print(f"Final Test Accuracy (Part 1 - Scratch CNN): {final_test_accuracy:.4f}")


Final Test Accuracy (Part 1 - Scratch CNN): 0.8340


In [25]:
from PIL import Image
import torchvision.transforms as transforms
import os
import random

# Pick random validation image
cat_images = os.listdir("dataset/val/cat")
random_image = random.choice(cat_images)

image_path = f"dataset/val/cat/{random_image}"
print("Testing image:", random_image)

image = Image.open(image_path).convert("RGB")

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

image = transform(image).unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    output = model(image)
    prediction = output.argmax(1).item()

print("Predicted Class:", train_data.classes[prediction])


Testing image: 550.jpg
Predicted Class: cat


In [26]:
torch.save(model.state_dict(), "part1_scratch_cnn.pth")
print("Part 1 model saved successfully.")


Part 1 model saved successfully.
